In [3]:
import dialogflow

In [26]:
! export GOOGLE_APPLICATION_CREDENTIALS="C:\Byte\IyD\Chatbots\dialogflow_v2_credentials\bytebot-faq-demo-1-c9f35fbb1595.json"

"export" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [ ]:
#Funcionó activando el billing, etc, ver esta página
#https://github.com/googleapis/dialogflow-python-client-v2

In [4]:
intents_client = dialogflow.IntentsClient()

In [7]:
parent = intents_client.project_agent_path("bytebot-faq-demo-1")

In [9]:
training_phrases = []

In [11]:
part = dialogflow.types.Intent.TrainingPhrase.Part(text="prueba_training_phrases_part")

In [12]:
part

text: "prueba_training_phrases_part"

In [13]:
training_phrase = dialogflow.types.Intent.TrainingPhrase(parts=[part])

In [14]:
training_phrase

parts {
  text: "prueba_training_phrases_part"
}

In [15]:
training_phrases.append(training_phrase)

In [16]:
text = dialogflow.types.Intent.Message.Text(text="message_texts")

In [17]:
text

text: "m"
text: "e"
text: "s"
text: "s"
text: "a"
text: "g"
text: "e"
text: "_"
text: "t"
text: "e"
text: "x"
text: "t"
text: "s"

In [18]:
message = dialogflow.types.Intent.Message(text=text)

In [19]:
message

text {
  text: "m"
  text: "e"
  text: "s"
  text: "s"
  text: "a"
  text: "g"
  text: "e"
  text: "_"
  text: "t"
  text: "e"
  text: "x"
  text: "t"
  text: "s"
}

In [20]:
intent = dialogflow.types.Intent(
        display_name="intent_desde_apiv2_python_client",
        training_phrases=training_phrases,
        messages=[message])

In [21]:
response = intents_client.create_intent(parent, intent)

In [22]:
response

name: "projects/bytebot-faq-demo-1/agent/intents/547501cf-8015-4ead-9c66-b38a513ce1c4"
display_name: "intent_desde_apiv2_python_client"
priority: 500000
messages {
  text {
    text: "m"
    text: "e"
    text: "s"
    text: "s"
    text: "a"
    text: "g"
    text: "e"
    text: "_"
    text: "t"
    text: "e"
    text: "x"
    text: "t"
    text: "s"
  }
}

In [23]:
print('Intent created: {}'.format(response))

Intent created: name: "projects/bytebot-faq-demo-1/agent/intents/547501cf-8015-4ead-9c66-b38a513ce1c4"
display_name: "intent_desde_apiv2_python_client"
priority: 500000
messages {
  text {
    text: "m"
    text: "e"
    text: "s"
    text: "s"
    text: "a"
    text: "g"
    text: "e"
    text: "_"
    text: "t"
    text: "e"
    text: "x"
    text: "t"
    text: "s"
  }
}



## Creación de la función que separa la oración

In [188]:
question = "como hago cualquier cosa en mi cuenta"
parameter = "sueldo"
parameterlen = len(parameter)

In [189]:
#Dividiendo la oración en 3 partes, cuya estructura sería la siguiente: 
#question[0] = "Primera parte de la pregunta"
#question[1] = "Segunda parte de la pregunta, contiene solo el parámetro el cual será considerado como producto"
#question[2] = "Tercera parte de la pregunta"

In [190]:
def split_question(question, parameter):
    question[:question.find(parameter)] + question[question.find(parameter):question.find(parameter) + parameterlen] + question[question.find(parameter) + parameterlen :]
    return [question[:question.find(parameter)], question[question.find(parameter):question.find(parameter) + parameterlen], question[question.find(parameter) + parameterlen :]]

In [191]:
split_question(question, parameter)

['como hago cualquier cosa en mi cuent',
 '',
 'hago cualquier cosa en mi cuenta']

#### Fin de la creación de la función que separa la oración

## Creación de intents mediante Dialogflow V2 - Original

La agregación de entities o parámetros cuando se crean las frases de entrenamiento no es automático, tiene que hacerse manual.

In [179]:
project_id = "bytebot-faq-demo-1"
display_name = "textintentv2_28"
training_phrases_parts = ["Qué es una cuenta {@producto:sueldo} bimoneda", "Dime que es una cuenta sueldo por favor"]
message_texts = ["Nosé papu", "Esto es una prueba :D", "Varias respuestas desde el API", "$producto"]

In [180]:
def create_intent(project_id, display_name, training_phrases_parts,
                  message_texts):
    """Create an intent of the given intent type."""
    intents_client = dialogflow.IntentsClient()

    parent = intents_client.project_agent_path(project_id)
    training_phrases = []
    for training_phrases_part in training_phrases_parts:
        
        part2 = dialogflow.types.Intent.TrainingPhrase.Part(
            text=training_phrases_part, entity_type="@producto", alias="producto", user_defined = True)

        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.types.Intent.TrainingPhrase(parts=[part2])
        training_phrases.append(training_phrase)

    text = dialogflow.types.Intent.Message.Text(text=message_texts)
    message = dialogflow.types.Intent.Message(text=text)

    intent = dialogflow.types.Intent(
        display_name=display_name,
        training_phrases=training_phrases,
        messages=[message])

    response = intents_client.create_intent(parent, intent)

    print('Intent created: {}'.format(response))

In [181]:
create_intent(project_id, display_name, training_phrases_parts,message_texts)

Intent created: name: "projects/bytebot-faq-demo-1/agent/intents/2724c91d-5477-4a1e-a520-7afd57912244"
display_name: "textintentv2_28"
priority: 500000
messages {
  text {
    text: "Nos\303\251 papu"
    text: "Esto es una prueba :D"
    text: "Varias respuestas desde el API"
    text: "$producto"
  }
}



## Creación de intents mediante Dialogflow V2 - parámetros en frase de entrenamiento manual

In [192]:
project_id = "bytebot-faq-demo-1"
display_name = "test_01"
training_phrases_parts = ["Qué es una cuenta sueldo bimoneda"]
message_texts = ["Nosé papu", "Esto es una prueba :D", "Varias respuestas desde el API", "$producto"]

In [194]:
def create_intent(project_id, display_name, training_phrases_parts,
                  message_texts):
    """Create an intent of the given intent type."""
    intents_client = dialogflow.IntentsClient()

    parent = intents_client.project_agent_path(project_id)
    training_phrases = []
    for training_phrases_part in training_phrases_parts:
        part1 = dialogflow.types.Intent.TrainingPhrase.Part(
            text=split_question(training_phrases_part, "sueldo")[0])
        
        part2 = dialogflow.types.Intent.TrainingPhrase.Part(
            text=split_question(training_phrases_part, "sueldo")[1], entity_type="@producto", alias="producto")
        
        part3 = dialogflow.types.Intent.TrainingPhrase.Part(
            text=split_question(training_phrases_part, "sueldo")[2])
        
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.types.Intent.TrainingPhrase(parts=[part1, part2, part3])
        training_phrases.append(training_phrase)

    text = dialogflow.types.Intent.Message.Text(text=message_texts)
    message = dialogflow.types.Intent.Message(text=text)

    intent = dialogflow.types.Intent(
        display_name=display_name,
        training_phrases=training_phrases,
        messages=[message],
        webhook_state=True)

    response = intents_client.create_intent(parent, intent)

    print('Intent created: {}'.format(response))

In [197]:
project_id = "bytebot-faq-demo-1"
display_name = "test_02"
training_phrases_parts = ["¿Mensualmente recibo estados de cuenta?"]
message_texts = ["Nosé papu", "Esto es una prueba :D", "Varias respuestas desde el API", "$producto"]

In [200]:
def create_intent_2(project_id, display_name, training_phrases_parts,
                  message_texts):
    """Create an intent of the given intent type."""
    intents_client = dialogflow.IntentsClient()

    parent = intents_client.project_agent_path(project_id)
    training_phrases = []
    for training_phrases_part in training_phrases_parts:
        part1 = dialogflow.types.Intent.TrainingPhrase.Part(
            text=training_phrases_part)
        
        
        
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.types.Intent.TrainingPhrase(parts=[part1])
        training_phrases.append(training_phrase)

    text = dialogflow.types.Intent.Message.Text(text=message_texts)
    message = dialogflow.types.Intent.Message(text=text)

    intent = dialogflow.types.Intent(
        display_name=display_name,
        training_phrases=training_phrases,
        messages=[message],
        webhook_state=True)

    response = intents_client.create_intent(parent, intent)

    print('Intent created: {}'.format(response))

In [195]:
create_intent(project_id, display_name, training_phrases_parts,message_texts)

Intent created: name: "projects/bytebot-faq-demo-1/agent/intents/bf0ef119-9a72-4f6f-b193-21b69e457791"
display_name: "test_01"
priority: 500000
webhook_state: WEBHOOK_STATE_ENABLED
messages {
  text {
    text: "Nos\303\251 papu"
    text: "Esto es una prueba :D"
    text: "Varias respuestas desde el API"
    text: "$producto"
  }
}



In [201]:
create_intent_2(project_id, display_name, training_phrases_parts,message_texts)

Intent created: name: "projects/bytebot-faq-demo-1/agent/intents/06021ee3-a86b-4bdc-beb0-41bafe047ce3"
display_name: "test_02"
priority: 500000
webhook_state: WEBHOOK_STATE_ENABLED
messages {
  text {
    text: "Nos\303\251 papu"
    text: "Esto es una prueba :D"
    text: "Varias respuestas desde el API"
    text: "$producto"
  }
}



In [205]:
queryResult =  {
    "queryText": "¿Mensualmente recibo estados de cuenta?",
    "parameters": {},
    "allRequiredParamsPresent": True,
    "fulfillmentMessages": [
      {
        "text": {
          "text": [
            ""
          ]
        }
      }
    ],
    "outputContexts": [
      {
        "name": "projects/bytebot-faq-demo-1/agent/sessions/30739716-36e5-8e8b-1758-584c5419e3f1/contexts/context_name",
        "lifespanCount": 3,
        "parameters": {
          "pregunta": "¿Mensualmente recibo estados de cuenta?"
        }
      },
      {
        "name": "projects/bytebot-faq-demo-1/agent/sessions/30739716-36e5-8e8b-1758-584c5419e3f1/contexts/requiere_parametro",
        "lifespanCount": 5
      }
    ],
    "intent": {
      "name": "projects/bytebot-faq-demo-1/agent/intents/bb3a59ba-58df-40ef-ac96-ecd45adb3949",
      "displayName": "intent.4.facil"
    },
    "intentDetectionConfidence": 0.6666667,
    "languageCode": "es"
  }

In [218]:
outputContexts = queryResult.get("outputContexts")
for element in outputContexts:
    if element.get("name").find("context_name"):
        pregunta = element.get("parameters").get("pregunta")
        break
        
pregunta
    

'¿Mensualmente recibo estados de cuenta?'

In [223]:
element.get("parameters").get("pregunsta") == None

True

In [210]:
intent = queryResult.get("intent")
intentDisplayName = intent.get("displayName")
intentDisplayName

intentDisplayName.find("unico")

-1

In [281]:
a = "intent_0.ganadora.unico"

In [270]:
import re



In [230]:
s = 'asdf=5;iwantthis123jasd'


In [272]:
result = re.search('intent(.*)unico', a)


In [274]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""


In [285]:
find_between(a,".",".unico")

'ganadora'

In [280]:
a.index( "." )

6